### Char-RNN(문자 단위) 사용해 셰익스피어 문체 생성하기

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

C:\Users\knuyh\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# 훈련 데이터셋 만들기
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

1115394/1115394 [==============================] - 0s 0us/step


In [3]:
# 모든 글자 정수로 인코딩
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True) # 단어 수준 인코딩 대신 글자 수준 인코딩, 기본적으로 소문자로 변경
tokenizer.fit_on_texts(shakespeare_text) # 빈도수 기준 단어 집합 생성

In [7]:
tokenizer.texts_to_sequences(["First"]) # F의 인덱스 20

[[20, 6, 9, 8, 3]]

In [9]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [10]:
max_id = len(tokenizer.word_index) # 고유한 문자 개수
dataset_size = tokenizer.document_count # 전체 문자 개수

In [11]:
max_id, dataset_size

(39, 1115394)

In [14]:
# 전체 텍스트 인코딩
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
train_size = dataset_size * 90 // 100 # 처음 90%를 훈련셋으로 사용 ; 1003854
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [18]:
train_size

1003854

In [20]:
# 윈도 여러개로 자르기
# TBPTT : window()로 긴 시퀀스를 작은 많은 텍스트 윈도로 변환하고, 부분 문자열 길이만큼만 역전파
n_steps = 100
window_length = n_steps + 1 # target = 1글자 앞의 input
dataset = dataset.window(window_length, shift=1, drop_remainder=True)
'''
shift = 1 : 가장 큰 훈련 셋 생성
drop_remainder = T
: 첫 번째 윈도 0 ~ 100
두 번째 윈도 1 ~ 101
패딩 없이 배치 데이터 만들기 위해 모든 윈도가 동일하게 101개의 글자 포함
if F일 경우, 윈도 100개는 글자 100개, 글자 99개 처럼 점점 줄어 마지막 윈도는 글자 1개
'''

'\nshift = 1 : 가장 큰 훈련 셋 생성\ndrop_remainder = T\n: 첫 번째 윈도 0 ~ 100\n두 번째 윈도 1 ~ 101\n패딩 없이 배치 데이터 만들기 위해 모든 윈도가 동일하게 101개의 글자 포함\nif F일 경우, 윈도 100개는 글자 100개, 글자 99개 처럼 점점 줄어 마지막 윈도는 글자 1개\n'

* 리스트의 리스트와 비슷한 중첩데이터셋 : 각 윈도를 변환할 때 유용,
* 하지만 모델은 데이터셋이 아니라 텐서를 기대하기에 훈련에 중첩 데이터셋 바로 사용 X
* 따라서 중첩 데이터셋을 플랫 데이터셋(데이터셋이 들어 있지 않는 데이터셋)으로 변환
------------
flat_map()  
{{1, 2}, {3, 4, 5, 6}} -> [1, 2, 3, 4, 5, 6]  
flat_map(lambda ds : ds.batch(2))  
{{1, 2}, {3, 4}, {5, 6}} 텐서 2개를 가진 데이터셋

In [23]:
dataset = dataset.flat_map(lambda window : window.batch(window_length))
# 윈도마다 batch(window_length) 호출 : 윈도 길이와 같기에 텐서 하나를 담은 데이터셋
# 연속된 101 글자 길이의 윈도

In [28]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [30]:
# 범주형 인코딩 (원핫벡터)
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [33]:
dataset = dataset.prefetch(1)

In [44]:
# 모델 만들고 훈련하기
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     #dropout=0.2, recurrent_dropout=0.2), # 입력, 은닉상태
                     dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=1) # epoch=10으로

31368/31368 [==============================] - 6131s 195ms/step - loss: 1.6196


In [47]:
# 전처리
def preprocess(texts) :
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [48]:
# 다음 글자 예측
X_new = preprocess(['How are yo'])
Y_pred = np.argmax(model(X_new), axis = -1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 첫번째 문장, 마지막 글자

'u'

##### 가짜 셰익스피어 텍스트 생성하기
* 예측한 다음 글자를 텍스트 끝에 추가하고, 늘어난 텍스트를 모델에 전달하여 다음 글자 예측하는 모델
* 하지만, 같은 단어가 계속 반복되는 경우 많음
* 대신 tf.random.categorical() 함수 사용해 모델이 추정한 확률을 기반으로 다음 글자 무작위 선택
* 생성된 텍스트의 다양성 많이 제어하기 위해 temperature 숫자로 로짓 나눔
    * 0에 가까울수록 높은 확률을 가진 글자 선택
    * 높으면 모든 글자가 동일한 확률 가짐

In [54]:
# 다음글자 선택하고, 입력 텍스트에 추가
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [55]:
next_char("How are yo", temperature=1)

'u'

In [56]:
# next_char 반복호출하여 다음 글자 얻고, 텍스트에 추가하는 함수
def complete_text(text, n_chars=50, temperature=1) :
    for _ in range(n_chars) :
        text += next_char(text, temperature)
    return text

In [57]:
print(complete_text("t", temperature=0.2))
print(complete_text("t", temperature=1))
print(complete_text("t", temperature=2))

therefore i say,
that i shall be your ben contrive 
trance it in slandle?

gremio:
o thas well deds arr
to sesse! makt detpyers:
so ha g! go you, ye, isere


### 상태가 있는 RNN
* 지금까지는 상태가 없는 RNN 사용
* 훈련 반복마다 은닉 상태를 0으로 초기화
* 타임 스텝마다 이 상태를 업데이트하고 마지막 타임 스텝 후에는 더 필요가 없기에 버림
-------------
* 상태가 있는 RNN : 훈련 배치를 처리한 후, 마지막 상태를 다음 훈련 배치의 초기 상태로 사용하여 역전파는 짧은 시퀀스에서 일어나지만, 모델이 장기간 패턴을 학습할 수 있다.
* 배치에 있는 각 입력 시퀀스가 이전 배치의 시퀀스가 끝난 시점에서 시작해야 한다.
* 연속적인 윈도가 같은 배치에 들어간다. 이 윈도가 끝난 지점부터 다음 배치가 계속되지 않는다.
    * 첫 번째 배치 : 윈도 1 ~ 32
    * 두 번째 배치 : 윈도 33 ~ 64
    * 각 배치의 첫 번째 윈도를 생각하면 1, 33 연속적이지 않다.
    * 해결 : 하나의 윈도를 갖는 배치를 만든다.
------------------
1. 순차적이고, 겹치지 않는 입력 시퀀스 만든다.
2. window메서드에서 shift = n_steps 사용
3. shuffle 메서드 호출해선 안된다.


In [58]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [63]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [64]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True, # 각 순환층 만들 때 stateful=T
                     #dropout=0.2, recurrent_dropout=0.2,
                     dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]), # 배치 크기 알아야 한다.(배치에 있는 입력 시퀀스 상태 보존해야하기 때문)
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [65]:
# epoch 끝마다 텍스트 다시 시작하기 전에 상태 재설정
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [66]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=50,
                    callbacks=[ResetStatesCallback()])
# 배치마다 샘플 하나

Epoch 1/50
313/313 [==============================] - 45s 135ms/step - loss: 2.6223
Epoch 2/50
313/313 [==============================] - 47s 150ms/step - loss: 2.2381
Epoch 3/50
313/313 [==============================] - 48s 154ms/step - loss: 2.1071
Epoch 4/50
313/313 [==============================] - 48s 153ms/step - loss: 2.0317
Epoch 5/50
313/313 [==============================] - 50s 161ms/step - loss: 1.9825
Epoch 6/50
313/313 [==============================] - 48s 152ms/step - loss: 1.9448
Epoch 7/50
313/313 [==============================] - 52s 165ms/step - loss: 1.9189
Epoch 8/50
313/313 [==============================] - 48s 154ms/step - loss: 1.8965
Epoch 9/50
313/313 [==============================] - 51s 164ms/step - loss: 1.8770
Epoch 10/50
313/313 [==============================] - 50s 160ms/step - loss: 1.8632
Epoch 11/50
313/313 [==============================] - 48s 153ms/step - loss: 1.8489
Epoch 12/50
313/313 [==============================] - 51s 161ms/step - lo

* 모델 훈련 후, 훈련할 때 사용한 것과 동일한 크기의 배치로만 예측 만들 수 있다.
* 이런 제약 없애려면, 동일한 구조의 상태가 '없는' 모델 만들고, 상태가 있는 모델의 가중치 복사

In [67]:
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [68]:
# 가중치 복사위해 먼저 모델 빌드
stateless_model.build(tf.TensorShape([None, None, max_id]))

stateless_model.set_weights(model.get_weights())
model = stateless_model

In [69]:
print(complete_text("t"))

tibre, forthing;
isabel, she's i cantothat his sunt


### 감성 분석

In [70]:
# IMDB 데이터셋 로드
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()

17464789/17464789 [==============================] - 2s 0us/step


In [71]:
X_train[0][:10] # 각 리뷰들의 리스트, 각 리뷰는 넘파이 정수 배열 : 각 정수는 하나의 단어, 빈도에 따라 인덱스(낮은 정수(0)=높은 빈도)

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [72]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")): 
    # 정수 0 - 패딩, 1 - SOS(start of sequence), 2 - UNK
    id_to_word[id_] = token
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

1641221/1641221 [==============================] - 0s 0us/step


'<sos> this film was just brilliant casting location scenery story'

In [73]:
# 전처리를 모델 자체에 포함
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits['train'].num_examples
test_size = info.splits["test"].num_examples
train_size, test_size

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\knuyh\tensorflow_datasets\imdb_reviews\plain_text\incomplete.ANYT9B_1.0.0\imdb_reviews-trai…

...

Shuffling C:\Users\knuyh\tensorflow_datasets\imdb_reviews\plain_text\incomplete.ANYT9B_1.0.0\imdb_reviews-test…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\knuyh\tensorflow_datasets\imdb_reviews\plain_text\incomplete.ANYT9B_1.0.0\imdb_reviews-unsu…

Dataset imdb_reviews downloaded and prepared to C:\Users\knuyh\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


(25000, 25000)

In [76]:
for X_batch, y_batch in datasets["train"].batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print("Review:", review.decode("utf-8")[:200], "...")
        print("Label:", label, "= Positive" if label else "= Negative")
        print()

Review: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0 = Negative

Review: I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0 = Negative



In [74]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300) # 각 리뷰에서 처음 300글자만 남김
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch # default_value 지정하지 않으면 빈 바이트 문자열로 패딩

In [77]:
preprocess(X_batch, y_batch)

(<tf.Tensor: shape=(2, 53), dtype=string, numpy=
 array([[b'This', b'was', b'an', b'absolutely', b'terrible', b'movie',
         b"Don't", b'be', b'lured', b'in', b'by', b'Christopher',
         b'Walken', b'or', b'Michael', b'Ironside', b'Both', b'are',
         b'great', b'actors', b'but', b'this', b'must', b'simply', b'be',
         b'their', b'worst', b'role', b'in', b'history', b'Even',
         b'their', b'great', b'acting', b'could', b'not', b'redeem',
         b'this', b"movie's", b'ridiculous', b'storyline', b'This',
         b'movie', b'is', b'an', b'early', b'nineties', b'US',
         b'propaganda', b'pi', b'<pad>', b'<pad>', b'<pad>'],
        [b'I', b'have', b'been', b'known', b'to', b'fall', b'asleep',
         b'during', b'films', b'but', b'this', b'is', b'usually', b'due',
         b'to', b'a', b'combination', b'of', b'things', b'including',
         b'really', b'tired', b'being', b'warm', b'and', b'comfortable',
         b'on', b'the', b'sette', b'and', b'having', b'j

In [78]:
# 어휘 사전 구축, 단어의 등장 횟수
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in datasets['train'].batch(32).map(preprocess) :
    for review in X_batch :
        vocabulary.update(list(review.numpy()))

In [79]:
vocabulary.most_common()[:3]

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

In [80]:
len(vocabulary)

53893

In [81]:
# 가장 많이 등장하는 단어 10000개만
vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]

In [84]:
# ID(인덱스)로 바꾸는 전처리 단계
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [85]:
word_to_id = {word: index for index, word in enumerate(truncated_vocabulary)}
for word in b"This movie was faaaaaantastic".split():
    print(word_to_id.get(word) or vocab_size)

22
12
11
10000


In [86]:
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10053]], dtype=int64)>

In [87]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].batch(32).map(preprocess) # 단어 -> 짧은 시퀀스
train_set = train_set.map(encode_words).prefetch(1) # 단어 인코딩

In [88]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True,
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 97s 114ms/step - loss: 0.5404 - accuracy: 0.7194
Epoch 2/5
782/782 [==============================] - 97s 124ms/step - loss: 0.3472 - accuracy: 0.8572
Epoch 3/5
782/782 [==============================] - 102s 130ms/step - loss: 0.1921 - accuracy: 0.9324
Epoch 4/5
782/782 [==============================] - 104s 133ms/step - loss: 0.1400 - accuracy: 0.9502
Epoch 5/5
782/782 [==============================] - 104s 133ms/step - loss: 0.1154 - accuracy: 0.9573


In [89]:
# 마스킹
# 패딩 토큰을 무시하도록 모델에게 알려주어 실제 의미가 있는 데이터에 집중할 수 있게
# Embedding층에서 mask_zero = True -> 모든 층(IC=0)에서 패딩 토큰 무시
K = keras.backend
embed_size = 128
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation="sigmoid")(z)
model = keras.models.Model(inputs=[inputs], outputs=[outputs])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 104s 124ms/step - loss: 0.5398 - accuracy: 0.7155
Epoch 2/5
782/782 [==============================] - 100s 127ms/step - loss: 0.3474 - accuracy: 0.8560
Epoch 3/5
782/782 [==============================] - 98s 126ms/step - loss: 0.1855 - accuracy: 0.9347
Epoch 4/5
782/782 [==============================] - 96s 123ms/step - loss: 0.1356 - accuracy: 0.9521
Epoch 5/5
782/782 [==============================] - 99s 126ms/step - loss: 0.1051 - accuracy: 0.9615


In [104]:
cache_dir = '/tmp/tfhub_cache'
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)
os.environ['TFHUB_CACHE_DIR'] = cache_dir

In [105]:
# 사전 훈련된 임베딩 재사용하기
import tensorflow_hub as hub

model = keras.Sequential([
    hub.KerasLayer(r"https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                   dtype=tf.string, input_shape=[], output_shape=[50]), # url에서 모듈(문장인코더) 다운로드
    # 문자열을 입력으로 받아 50차원의 하나의 벡터로 인코딩
    # hub.KerasLayer 층은 훈련되지 않음, trainable=True 설정하면 작업에 맞게 미세조정 가능
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation = 'sigmoid')
])

In [106]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [107]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = datasets["train"].batch(batch_size).prefetch(1)
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 3s 3ms/step - loss: 0.5489 - accuracy: 0.7235
Epoch 2/5
782/782 [==============================] - 2s 3ms/step - loss: 0.5142 - accuracy: 0.7487
Epoch 3/5
782/782 [==============================] - 2s 3ms/step - loss: 0.5089 - accuracy: 0.7510
Epoch 4/5
782/782 [==============================] - 2s 3ms/step - loss: 0.5052 - accuracy: 0.7550
Epoch 5/5
782/782 [==============================] - 2s 3ms/step - loss: 0.5023 - accuracy: 0.7558


### 신경망 기계 번역을 위한 인코더-디코더 네트워크
* 자동 번역

tensorflow_addons 설치 오류 이슈

In [108]:
vocab_size = 100
embed_size = 10

In [ ]:
from tensorflow import keras
import tensorflow_addons as tfa # 시퀀스 투 시퀀스 도구를 가지고 있음

encoder_inputs = tensorflow.keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder = keras.layers.LSTM(512, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler() # 각 스텝에서 디코더에게 이전 스텝의 출력이 무엇인지 알려줌
# 훈련 시, 이전 타깃 토큰의 임베딩
# 추론 시, 실제로 출력되는 토큰의 임베딩

decoder_cell = keras.layers.LSTMCell(512)
output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler,
                                                 output_layer=output_layer)

final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings, initial_state=encoder_state,
    sequence_length=sequence_lengths)
Y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.models.Model(
    inputs=[encoder_inputs, decoder_inputs, sequence_lengths],
    outputs=[Y_proba])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [ ]:
X = np.random.randint(100, size=10*1000).reshape(1000, 10)
Y = np.random.randint(100, size=15*1000).reshape(1000, 15)
X_decoder = np.c_[np.zeros((1000, 1)), Y[:, :-1]]
seq_lengths = np.full([1000], 15)

history = model.fit([X, X_decoder, seq_lengths], Y, epochs=2)

In [130]:
# 양방향 순환층
# 층을 복사(반대방향으로), 그다음 두 층을 실행하여 그 출력 연결
# ex. GRU 층이 10개 유닛을 가지면, 양방향 층은 타임 스텝마다 20개 값 출력

model = keras.models.Sequential([
    keras.layers.GRU(10, return_sequences=True, input_shape=[None, 10]),
    keras.layers.Bidirectional(keras.layers.GRU(10, return_sequences=True))
])

model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_24 (GRU)                (None, None, 10)          660       
                                                                 
 bidirectional (Bidirection  (None, None, 20)          1320      
 al)                                                             
                                                                 
Total params: 1980 (7.73 KB)
Trainable params: 1980 (7.73 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# 빔 검색 : 모델이 앞선 실수를 고칠 수 있게 하는 방법
# k(빔 너비)개의 가능성 있는 문장의 리스트 유지하고, 디코더 단계마다 이 문장의 단어를 하나씩 생성하여 가능성 있는 k개의 문장 만듦
beam_width = 10
decoder = tfa.seq2seq.beam_search_decoder.BeamSearchDecoder(
cell=decoder_cell, beam_width=beam_width, output_layer=output_layer)

decoder_initial_state=tfa.seq2seq.beam_search_decoder.tile_batch(
encoder_state, multiplier=beam_width) # 인코더의 마지막 상태 복사
outputs, _, _ = decoder(embedding_decoder, start_tokens=start_tokens,
                       end_token = end_token, initial_state=decoder_initial_state)

### 어텐션 메커니즘
#### 인코더
* 단어 ID의 시퀀스로 표현된 문장의 배치를 입력으로 받음 (입력 크기 : [배치크기, 입력 문장의 최대 길이])
* 각 단어를 512차원의 표현으로 인코딩 (출력 크기 : [배치 크기, 입력 문장의 최대 길이, 512])

    1) 인코더의 멀티-헤드 어텐션
    * 관련이 많은 단어에 더 많은 주의를 기울이면서 각 단어와 동일한 문장에 있는 다른 단어와의 관계 인코딩
     * ex) They welcomed the Queen of the United Kingdom -> Queen은 모든 단어에 의존적이지만, They나 welcomed 보다 United, Kingdom에 더 주의를 기울인다.
     => **self- attention**
#### 디코더
* 훈련하는 동안 타깃 문장을 입력으로 받음
    * 이 입력은 오른쪽으로 한 타임 스텝 이동되어 있음
* 인코더의 출력을 받음
* 디코더의 출력 크기 : [배치 크기, 출력 문장의 최대 길이, 어휘 사전 길이]
* 추론 시에는 디코더에 타깃 주입할 수 없음

    1) 디코더의 마스크드 멀티-헤드 어텐션
    * 인코더의 멀티 헤드 어텐션과 동일한 작업 수행하지만, 각 단어는 이전에 등장한 단어에만 주의를 기울일 수 있다.
    2) 디코더의 위쪽 멀티-헤드 어텐션 층
    * 디코더가 입력 문장에 있는 단어에 주의를 기울이는 곳

----------------------------
* 위치 인코딩 : 문장에 있는 단어의 위치를 나타내는 단순한 밀집 벡터
    * 멀티-헤드 어텐션 층이 단어 사이 관계만 보고, 단어의 순서나 위치를 고려하지 않기에 필요함

In [133]:
class PositionalEncoding(keras.layers.Layer):
    def __init__(self, max_steps, max_dims, dtype=tf.float32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        if max_dims % 2 == 1: max_dims += 1 # max_dims 는 짝수여야
        p, i = np.meshgrid(np.arange(max_steps), np.arange(max_dims // 2))
        pos_emb = np.empty((1, max_steps, max_dims))
        pos_emb[0, :, ::2] = np.sin(p / 10000**(2 * i / max_dims)).T
        pos_emb[0, :, 1::2] = np.cos(p / 10000**(2 * i / max_dims)).T
        self.positional_embedding = tf.constant(pos_emb.astype(self.dtype))
        
    def call(self, inputs):  # 인코딩 행렬의 입력의 크기로 잘라 입력에 더함
        shape = tf.shape(inputs)
        return inputs + self.positional_embedding[:, :shape[-2], :shape[-1]]

In [134]:
max_steps = 201
max_dims = 512
pos_emb = PositionalEncoding(max_steps, max_dims)
PE = pos_emb(np.zeros((1, max_steps, max_dims), np.float32))[0].numpy()

In [135]:
embed_size = 512; max_steps = 500; vocab_size = 10000
encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)
positional_encoding = PositionalEncoding(max_steps, max_dims=embed_size)
encoder_in = positional_encoding(encoder_embeddings)
decoder_in = positional_encoding(decoder_embeddings)

* 멀티-헤드 어텐션

In [136]:
Z = encoder_in
for N in range(6):
    Z = keras.layers.Attention(use_scale=True)([Z, Z]) # use_scale : 파라미터 추가되어 유사도 점수의 스케일 적절히 낮추는 방법

encoder_outputs = Z
Z = decoder_in
for N in range(6):
    Z = keras.layers.Attention(use_scale=True, causal=True)([Z, Z]) # causal : 각 출력 토큰은 미래 토큰이 아닌 이전 출력 토큰에만 주의를 기울임
    Z = keras.layers.Attention(use_scale=True)([Z, encoder_outputs])

outputs = keras.layers.TimeDistributed(
    keras.layers.Dense(vocab_size, activation="softmax"))(Z)

In [137]:
K = keras.backend

class MultiHeadAttention(keras.layers.Layer):
    def __init__(self, n_heads, causal=False, use_scale=False, **kwargs):
        self.n_heads = n_heads
        self.causal = causal
        self.use_scale = use_scale
        super().__init__(**kwargs)
    def build(self, batch_input_shape):
        self.dims = batch_input_shape[0][-1]
        self.q_dims, self.v_dims, self.k_dims = [self.dims // self.n_heads] * 3 # could be hyperparameters instead
        self.q_linear = keras.layers.Conv1D(self.n_heads * self.q_dims, kernel_size=1, use_bias=False)
        self.v_linear = keras.layers.Conv1D(self.n_heads * self.v_dims, kernel_size=1, use_bias=False)
        self.k_linear = keras.layers.Conv1D(self.n_heads * self.k_dims, kernel_size=1, use_bias=False)
        self.attention = keras.layers.Attention(causal=self.causal, use_scale=self.use_scale)
        self.out_linear = keras.layers.Conv1D(self.dims, kernel_size=1, use_bias=False)
        super().build(batch_input_shape)
    def _multi_head_linear(self, inputs, linear):
        shape = K.concatenate([K.shape(inputs)[:-1], [self.n_heads, -1]])
        projected = K.reshape(linear(inputs), shape)
        perm = K.permute_dimensions(projected, [0, 2, 1, 3])
        return K.reshape(perm, [shape[0] * self.n_heads, shape[1], -1])
    def call(self, inputs):
        q = inputs[0]
        v = inputs[1]
        k = inputs[2] if len(inputs) > 2 else v
        shape = K.shape(q)
        q_proj = self._multi_head_linear(q, self.q_linear)
        v_proj = self._multi_head_linear(v, self.v_linear)
        k_proj = self._multi_head_linear(k, self.k_linear)
        multi_attended = self.attention([q_proj, v_proj, k_proj])
        shape_attended = K.shape(multi_attended)
        reshaped_attended = K.reshape(multi_attended, [shape[0], self.n_heads, shape_attended[1], shape_attended[2]])
        perm = K.permute_dimensions(reshaped_attended, [0, 2, 1, 3])
        concat = K.reshape(perm, [shape[0], shape_attended[1], -1])
        return self.out_linear(concat)

In [138]:
Q = np.random.rand(2, 50, 512)
V = np.random.rand(2, 80, 512)
multi_attn = MultiHeadAttention(8)
multi_attn([Q, V]).shape

TensorShape([2, 50, 512])